<a href="https://colab.research.google.com/github/tsdeepak/cnn-mnist/blob/main/s6_cnn_mnsit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

 !pip install --upgrade torch torchvision torchinfo tqdm matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchinfo import summary

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
torch.manual_seed(42)
batch_size = 64

kwargs = {'num_workers': 3, 'pin_memory': True} if device != 'cpu' else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=False, **kwargs)

100%|██████████| 9912422/9912422 [00:00<00:00, 91542581.70it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 36311658.82it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25124667.67it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3062293.65it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:

DROP = 0.05

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnv_block1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1, bias=False),    # Input -  28x28x1, Output -  28x28x8, RF - 3x3
            nn.Dropout(DROP),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 8, 3, padding=1, bias=False),    # Input -  28x28x8, Output -  28x28x8, RF - 5x5
            nn.Dropout(DROP),
            nn.ReLU()
        )

        self.t_block1 = nn.Sequential(
            nn.MaxPool2d(2, 2)                            # Input -  28x28x8, Output -  14x14x8, RF - 6x6
        )

        self.cnv_block2 = nn.Sequential(
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, 3, padding=1, bias=False),   # Input -  14x14x8, Output - 14x14x16, RF - 10x10
            nn.Dropout(DROP),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1, bias=False),  # Input - 14x14x16, Output - 14x14x16, RF - 14x14
            nn.Dropout(DROP),
            nn.ReLU()
        )

        self.t_block2 = nn.Sequential(
            nn.MaxPool2d(2, 2)                            # Input - 14x14x16, Output -   7x7x16, RF - 16x16
        )

        self.cnv_block3 = nn.Sequential(
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, 3, padding=1, bias=False),  # Input -   7x7x16, Output -   7x7x32, RF - 24x24
            nn.Dropout(DROP),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 3, padding=1, bias=False),  # Input -   7x7x32, Output -   7x7x32, RF - 32x32
            nn.Dropout(DROP),
            nn.ReLU()
        )

        self.out_block = nn.Sequential(
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 1, bias=False),             # Input -   7x7x32, Output -   7x7x32, RF - 32x32
            nn.Dropout(DROP),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 10, 1),                         # Input -   7x7x32, Output -   7x7x10, RF - 32x32
            nn.ReLU(),
            nn.AvgPool2d(7, 7),                           # Input -   7x7x10, Output -   1x1x10
            nn.Flatten(),
            nn.LogSoftmax()
        )

    def forward(self, x):
        x = self.cnv_block1(x)
        x = self.t_block1(x)
        x = self.cnv_block2(x)
        x = self.t_block2(x)
        x = self.cnv_block3(x)
        x = self.out_block(x)
        return x
     

In [ ]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
           Dropout-2            [-1, 8, 28, 28]               0
              ReLU-3            [-1, 8, 28, 28]               0
       BatchNorm2d-4            [-1, 8, 28, 28]              16
            Conv2d-5            [-1, 8, 28, 28]             576
           Dropout-6            [-1, 8, 28, 28]               0
              ReLU-7            [-1, 8, 28, 28]               0
         MaxPool2d-8            [-1, 8, 14, 14]               0
       BatchNorm2d-9            [-1, 8, 14, 14]              16
           Conv2d-10           [-1, 16, 14, 14]           1,152
          Dropout-11           [-1, 16, 14, 14]               0
             ReLU-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
           Conv2d-14           [-1, 16,

100%|██████████| 9912422/9912422 [00:00<00:00, 95904628.13it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 38382665.98it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27243685.90it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7785259.00it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'Train set: Average loss={loss.item()} batch_id={batch_idx}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return test_loss

In [ ]:


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True)

for epoch in range(0, 20):
    print("Epoch {}: ".format(epoch + 1))
    train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)
    scheduler.step(test_loss)

Epoch 1: 


  0%|          | 0/938 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Train set: Average loss=0.12361396104097366 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 42.48it/s]



Test set: Average loss: 0.1003, Accuracy: 9702/10000 (97.02%)

Epoch 2: 


Train set: Average loss=0.012935614213347435 batch_id=937: 100%|██████████| 938/938 [00:21<00:00, 44.01it/s]



Test set: Average loss: 0.0819, Accuracy: 9748/10000 (97.48%)

Epoch 3: 


...


Test set: Average loss: 0.0271, Accuracy: 9918/10000 (99.18%)

Epoch 19: 


Train set: Average loss=0.015343899838626385 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 42.21it/s]



Test set: Average loss: 0.0296, Accuracy: 9908/10000 (99.08%)

Epoch 20: 


Train set: Average loss=0.006899966858327389 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 41.77it/s]



Test set: Average loss: 0.0330, Accuracy: 9896/10000 (98.96%)

Epoch 00020: reducing learning rate of group 0 to 1.0000e-08.
